# amount spent 예측

In [2]:
import pandas as pd
import numpy as np


In [22]:
x_raw = pd.read_csv('all_file_merge.csv')
x = x_raw 

In [23]:
from IPython.display import display
pd.options.display.max_columns = None
display(x.head())
# x.fillna(0, inplace=True)

Unnamed: 0  acc_id  survival_time  amount_spent   playtime   npc_kill  \
0           0   27835           64.0      0.002559   9.854158  12.769067   
1           1   12351           64.0      0.120154  80.993692  37.118665   
2           2  125437           55.0      0.182593  39.552392  13.483810   
3           3  104483           64.0      0.016241  60.901508  48.688721   
4           4    4704           20.0      0.226396  25.679328  17.134915   

   solo_exp  party_exp  quest_exp  rich_monster      death     revive  \
0  0.130705   0.000000   0.022557           1.0   0.245883   0.247337   
1  0.533964   0.039494   0.016483           7.0   0.983534   0.989348   
2  0.176769   0.015612   0.134217           7.0  12.048290  12.119508   
3  0.243562   0.001155   0.006941           2.0   1.475301   1.484021   
4  0.433204   0.546727   0.227314          15.0   0.737650   0.742011   

   exp_recovery    fishing  private_shop  game_money_change  enchant_count  \
0      0.000000   1.184645      0.000000           0.011269            0.0   
1      0.000000  41.419533      0.000000           1.591701            0.0   
2      0.000000  23.852057      0.000000          -0.061119            0.0   
3     31.897467   2.018737      0.114224           0.087539            0.0   
4      0.000000   9.309176      0.000000          -0.058276            0.0   

   level  pledge_cnt  random_attacker_cnt  random_defender_cnt  temp_cnt  \
0   17.0    0.000000                  0.0             0.000000  1.918911   
1   17.0    7.124451                  0.0             4.732258  5.756733   
2   17.0    2.631554                  0.0             2.366129  0.000000   
3   17.0    0.000000                  0.0            16.562903  1.918911   
4   16.0    1.026948                  0.0             3.549193  3.837822   

   same_pledge_cnt   etc_cnt  num_opponent  amount_spent_pay  sell_item_cnt  \
0         0.000000  2.139619      1.962575          0.873062            0.0   
1         0.000000  2.364842     12.854867          0.528062            5.0   
2         0.000000  4.054015      6.770884          1.815358           18.0   
3         0.000000  2.139619      2.747605          0.371990            1.0   
4        12.398273  3.603569      5.298953          1.404644            8.0   

   buy_item_cnt  play_char_cnt  combat_char_cnt  pledge_combat_cnt  \
0           0.0       0.000000         0.000000           0.000000   
1          20.0      28.402294         6.556226           3.782773   
2          44.0      50.380690        29.701510           8.097357   
3          20.0      74.043254         8.180246           0.369717   
4          39.0       6.649457         2.526252           0.100853   

   random_attacker_cnt_plg  random_defender_cnt_plg  same_pledge_cnt_plg  \
0                 0.000000                 0.000000             0.000000   
1                 2.804439                 1.867763            11.316867   
2                18.106920                 3.218910            43.764446   
3                 4.109830                 3.195066             7.426694   
4                 0.439763                 1.502158             1.094111   

   temp_cnt_plg  etc_cnt_plg  combat_play_time  non_combat_play_time  
0      0.000000     0.000000          0.000000              0.000000  
1     11.085888     3.106572         21.074054              0.019254  
2     27.291721    35.660301         79.411095              0.000000  
3     17.930793     1.802598         36.301640              0.000000  
4      7.960085     1.635964          7.578028              0.042944

## 전처리

In [9]:
import numpy as np

# 정규 분포를 이용하여 데이터 분포에 타원을 그립니다. 타원에서 벗어날수록 outlier
from sklearn.covariance import EllipticEnvelope
# 랜덤 데이터를 생성하는데 사용됩니다.
from sklearn.datasets import make_blobs
#https://gaussian37.github.io/ml-sklearn-Detecting-Outliers/

### 이상치 제거 


1	One-class SVM	                
Novelty

#### 2	Fitting an elliptic envelope	
Outlier

3	Isolation Forest	            
Outlier

4	Local Outlier Factor	        
Outlier


ref: https://mkjjo.github.io/python/2019/01/10/outlier.html


In [24]:
outlier_detector = EllipticEnvelope(contamination=.1)
outlier_detector.fit(x)
outl = pd.DataFrame(outlier_detector.predict(x))


C:\Users\CPB06GameN\Anaconda3\lib\site-packages\sklearn\covariance\robust_covariance.py:171: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (109.711557669397905 > 109.173778946857823). You may want to try with a higher value of support_fraction (current value: 0.501).
  RuntimeWarning)


In [25]:
x= pd.concat([x,outl],axis=1)
x =x.rename(columns={0:'outlier'})
x=x[x['outlier']==1]

In [26]:
# x = x[x['amount_spent_pay']>0] # 과금 유저만.

In [27]:
x.count()

Unnamed: 0                 36000
acc_id                     36000
survival_time              36000
amount_spent               36000
playtime                   36000
npc_kill                   36000
solo_exp                   36000
party_exp                  36000
quest_exp                  36000
rich_monster               36000
death                      36000
revive                     36000
exp_recovery               36000
fishing                    36000
private_shop               36000
game_money_change          36000
enchant_count              36000
level                      36000
pledge_cnt                 36000
random_attacker_cnt        36000
random_defender_cnt        36000
temp_cnt                   36000
same_pledge_cnt            36000
etc_cnt                    36000
num_opponent               36000
amount_spent_pay           36000
sell_item_cnt              36000
buy_item_cnt               36000
play_char_cnt              36000
combat_char_cnt            36000
pledge_com

In [36]:
# x = x[(x['amount_spent_pay']>np.percentile(x[['amount_spent_pay']], 25))& (x['amount_spent_pay']<np.percentile(x[['amount_spent_pay']], 75))] # 2-3분위

In [28]:
x[x['amount_spent_pay']<0].count() # payment 동일 유저.

Unnamed: 0                 0
acc_id                     0
survival_time              0
amount_spent               0
playtime                   0
npc_kill                   0
solo_exp                   0
party_exp                  0
quest_exp                  0
rich_monster               0
death                      0
revive                     0
exp_recovery               0
fishing                    0
private_shop               0
game_money_change          0
enchant_count              0
level                      0
pledge_cnt                 0
random_attacker_cnt        0
random_defender_cnt        0
temp_cnt                   0
same_pledge_cnt            0
etc_cnt                    0
num_opponent               0
amount_spent_pay           0
sell_item_cnt              0
buy_item_cnt               0
play_char_cnt              0
combat_char_cnt            0
pledge_combat_cnt          0
random_attacker_cnt_plg    0
random_defender_cnt_plg    0
same_pledge_cnt_plg        0
temp_cnt_plg  

In [29]:
x = x.drop(columns=['survival_time','Unnamed: 0','acc_id'])


In [30]:
len(x.index.values)

36000

In [31]:
x.count()

amount_spent               36000
playtime                   36000
npc_kill                   36000
solo_exp                   36000
party_exp                  36000
quest_exp                  36000
rich_monster               36000
death                      36000
revive                     36000
exp_recovery               36000
fishing                    36000
private_shop               36000
game_money_change          36000
enchant_count              36000
level                      36000
pledge_cnt                 36000
random_attacker_cnt        36000
random_defender_cnt        36000
temp_cnt                   36000
same_pledge_cnt            36000
etc_cnt                    36000
num_opponent               36000
amount_spent_pay           36000
sell_item_cnt              36000
buy_item_cnt               36000
play_char_cnt              36000
combat_char_cnt            36000
pledge_combat_cnt          36000
random_attacker_cnt_plg    36000
random_defender_cnt_plg    36000
same_pledg

In [32]:
x_data = [ x.iloc[i,1:-1].tolist() for i in range(len(x.index.values))]

In [33]:
x_data

[[9.854158463589807,
  12.769066855053186,
  0.13070541671142138,
  0.0,
  0.02255738021312401,
  1.0,
  0.2458834656477353,
  0.24733689711011106,
  0.0,
  1.1846446162463642,
  0.0,
  0.011269041855281029,
  0.0,
  17.0,
  0.0,
  0.0,
  0.0,
  1.9189110569779195,
  0.0,
  2.1396189985508025,
  1.9625750905578725,
  0.8730617954718154,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [80.99369247401384,
  37.11866521702612,
  0.5339639957376157,
  0.039493972545042735,
  0.016483377784727908,
  7.0,
  0.9835338625909412,
  0.9893475884404442,
  0.0,
  41.41953252355013,
  0.0,
  1.5917006416170536,
  0.0,
  17.0,
  7.124450931091294,
  0.0,
  4.732257939520743,
  5.756733170933758,
  0.0,
  2.3648420510298345,
  12.854866843154063,
  0.5280615698418238,
  5.0,
  20.0,
  28.402294328847017,
  6.5562263486749774,
  3.7827733970406,
  2.8044387805854822,
  1.8677627359825,
  11.316866840938628,
  11.085888360546141,
  3.1065719873107707,
  21.07405444

In [34]:
X = x_data

In [35]:
y_data = [[i] for i in x['amount_spent'].tolist() ]

y= y_data

## test, train 분할

In [37]:
from sklearn.model_selection import train_test_split

x_data, X_test, y_data, y_test = train_test_split(X, y, test_size = 1/3, random_state = 70) #42

## Scaler

1	StandardScaler	
기본 스케일. 평균과 표준편차 사용


#### 2	MinMaxScaler	
최대/최소값이 각각 1, 0이 되도록 스케일링


3	MaxAbsScaler	
최대절대값과 0이 각각 1, 0이 되도록 스케일링


4	RobustScaler	
중앙값(median)과 IQR(interquartile range) 사용. 아웃라이어의 영향을 최소화


#### MinMaxScaler

In [38]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(x_data)
data = scaler.transform(x_data)
x_data = data

In [39]:
scaler.fit(X_test)
data = scaler.transform(X_test)
X_test = data

#### RobustScaler

https://mkjjo.github.io/python/2019/01/10/scaler.html

In [49]:
# from sklearn.preprocessing import RobustScaler
# robustScaler = RobustScaler()
# print(robustScaler.fit(x_data))
# train_data_robustScaled = robustScaler.transform(x_data)
# x_data = train_data_robustScaled

In [50]:
# print(robustScaler.fit(X_test))
# train_data_robustScaled = robustScaler.transform(X_test)
# X_test = train_data_robustScaled

In [40]:
x_data = np.array(x_data)

In [41]:
x_data.shape

(24000, 34)

In [42]:
len(y_data)

24000

## Model (multi_variable_linear_regression)

In [43]:
# multi_variable_linear_regression.py
# 4

import tensorflow as tf


X = tf.placeholder(tf.float32,shape=[None,x_data.shape[1]])
Y = tf.placeholder(tf.float32,shape=[None,1])

W = tf.Variable(tf.random_normal([x_data.shape[1],1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

hypothesis = tf.matmul(X,W) + b  #

cost = tf.reduce_mean(tf.square(hypothesis - Y))

optimizer = tf.train.AdamOptimizer(learning_rate=3e-1)
train = optimizer.minimize(cost)



# RMSE( root-mean-square error ) : 평균 제곱 근 오차
# 평균 제곱근편차(RMSD) 또는 평균 제곱근오차(RMSE)는 추정값 또는 모델이 예측한 값과
# 실제 환경에서 관찰되는 값의 차이를 다룰 때 흔히 사용하는 측도이다.
# 정밀도를 표현하는데 적합하다.

targets = tf.placeholder(tf.float32,[None,1])  # 답(Y)
predictions = tf.placeholder(tf.float32,[None,1]) # 예측 값
rmse = tf.sqrt(tf.reduce_mean(tf.square(targets-predictions)))

sess = tf.Session()
sess.run(tf.global_variables_initializer())

# start training
for step in range(10001):
    cost_val, W_val, b_val, _ = \
        sess.run([cost, W, b, train],
                 feed_dict={X:x_data, Y:y_data})
    if step % 1000 == 0:
        print(step, cost_val, W_val, b_val)

# Test step
test_predict = sess.run(hypothesis,feed_dict={X:X_test})
rmse_val = sess.run(rmse,feed_dict={targets:y_test,predictions:test_predict})
print('RMSE: {}'.format(rmse_val))


total_error = tf.reduce_sum(tf.square(tf.subtract(y_test, tf.reduce_mean(y_test))))
unexplained_error = tf.reduce_sum(tf.square(tf.subtract(y_test, test_predict)))
R_squared = tf.subtract(1.0,tf.divide( unexplained_error,total_error))

print('R_sqr: {}'.format(sess.run(R_squared)))

prd = sess.run(hypothesis, feed_dict = {X:X_test})


0 3.9273915 [[-0.14556143]
 [-0.32052094]
 [ 2.474738  ]
 [ 0.4981327 ]
 [-1.9393581 ]
 [-0.37546456]
 [-0.3195961 ]
 [ 2.0707383 ]
 [-0.05262336]
 [ 0.5293511 ]
 [ 0.6067958 ]
 [-0.36336035]
 [ 0.21226045]
 [-0.02283111]
 [-0.9381031 ]
 [-0.1753459 ]
 [ 0.71252364]
 [ 0.6006069 ]
 [ 0.6466186 ]
 [ 1.4735482 ]
 [ 0.8434539 ]
 [ 0.0681529 ]
 [ 0.9353628 ]
 [-0.04943991]
 [ 1.5349567 ]
 [ 0.22860727]
 [-0.538697  ]
 [ 1.733441  ]
 [ 0.81687355]
 [ 2.0040703 ]
 [ 0.9689502 ]
 [ 0.9263673 ]
 [-1.4074295 ]
 [ 0.30705634]] [-0.88499475]
1000 0.60254574 [[-0.1757985 ]
 [-0.09124611]
 [-0.16169424]
 [-0.07621479]
 [-0.06211968]
 [-0.157218  ]
 [-0.09858162]
 [ 0.13485141]
 [-0.16669093]
 [ 0.30191085]
 [-0.01484166]
 [ 0.01228469]
 [-0.00078482]
 [-0.01099795]
 [ 0.14926821]
 [ 0.14605916]
 [-0.23249848]
 [ 0.04946488]
 [ 0.15957788]
 [ 0.261519  ]
 [-0.35476074]
 [ 0.53175545]
 [ 0.06753293]
 [-0.03521131]
 [ 0.10487911]
 [ 0.39823574]
 [ 0.1734811 ]
 [-0.01244008]
 [ 0.3951067 ]
 [ 0.0436498

In [46]:
col = list(x.columns)

col.pop(-1)
col

['playtime',
 'npc_kill',
 'solo_exp',
 'party_exp',
 'quest_exp',
 'boss_monster',
 'death',
 'revive',
 'exp_recovery',
 'fishing',
 'private_shop',
 'game_money_change',
 'enchant_count',
 'level',
 'pledge_cnt',
 'random_attacker_cnt',
 'random_defender_cnt',
 'temp_cnt',
 'same_pledge_cnt',
 'etc_cnt',
 'num_opponent',
 'amount_spent_pay',
 'sell_item_cnt',
 'buy_item_cnt',
 'play_char_cnt',
 'combat_char_cnt',
 'pledge_combat_cnt',
 'random_attacker_cnt_plg',
 'random_defender_cnt_plg',
 'same_pledge_cnt_plg',
 'temp_cnt_plg',
 'etc_cnt_plg',
 'combat_play_time',
 'non_combat_play_time']

In [205]:
w = pd.DataFrame(W_val)
w = w.rename(columns = {0: 'w'})
c = pd.DataFrame(col)
c = c.rename(columns = {0: 'colName'})

In [206]:
result = pd.concat([c,w],axis=1)
result

colName         w
0  random_attacker_cnt  0.334212
1              etc_cnt  0.010443
2  same_pledge_cnt_plg  0.307046

In [164]:
reTry= list(result[result['w']<0].colName)
reTry # 제외할 컬럼들

['sell_item_cnt', 'combat_play_time']

In [20]:
prd

array([[0.40206873],
       [0.43975586],
       [0.21693742],
       ...,
       [0.5914906 ],
       [0.47674537],
       [0.42076862]], dtype=float32)

In [60]:
pd.options.display.float_format = '{:.7f}'.format
#https://financedata.github.io/posts/pandas-display-format.html
#pandas 실수방식으로 표현

In [61]:
from IPython.display import display
pd.options.display.max_rows = None

display(pd.concat([pd.DataFrame(prd),pd.DataFrame(y_test)],axis=1))




0          0
0      0.0322988  0.0360922
1      0.1996944  0.2231479
2      0.0000000  0.0000000
3      0.0446898  0.0499384
4      0.0000000  0.0000000
5      0.0963971  0.1077187
6      0.1337463  0.1494543
7      0.0220219  0.0246083
8      0.0009102  0.0010171
9      0.0000000  0.0000000
10     0.0000000  0.0000000
11     0.0000000  0.0000000
12     0.0322988  0.0360922
13     0.0000000  0.0000000
14     0.3352317  0.3746037
15     0.0370261  0.0413747
16     0.0000000  0.0000000
17     0.0590774  0.0660158
18     0.0000000  0.0000000
19     0.0230202  0.0257239
20     0.4555886  0.5090962
21     0.0543794  0.0607661
22     0.0400505  0.0447543
23     0.0322988  0.0360922
24     0.1813429  0.2026410
25     0.0869718  0.0971863
26     0.0000000  0.0000000
27     0.0593416  0.0663111
28     0.0272484  0.0304487
29     0.0000000  0.0000000
30     0.0000000  0.0000000
31     0.0000000  0.0000000
32     0.0310656  0.0347141
33     0.0000000  0.0000000
34     0.2268548  0.2534982
35     0.0000000  0.0000000
36     0.0000000  0.0000000
37     0.0322988  0.0360922
38     0.0000000  0.0000000
39     0.3999173  0.4468865
40     0.0000000  0.0000000
41     0.2082389  0.2326960
42     0.0660657  0.0738249
43     0.0000000  0.0000000
44     0.0000000  0.0000000
45     0.0000000  0.0000000
46     0.0211410  0.0236240
47     0.0000000  0.0000000
48     0.0198197  0.0221475
49     0.0299498  0.0334673
50     0.0000000  0.0000000
51     0.0000000  0.0000000
52     0.0000000  0.0000000
53     0.0773409  0.0864243
54     0.0686789  0.0767450
55     0.0000000  0.0000000
56     0.0000000  0.0000000
57     0.1799335  0.2010661
58     0.4266372  0.4767445
59     0.0000000  0.0000000
60     0.1935283  0.2162576
61     0.0000000  0.0000000
62     0.0000000  0.0000000
63     0.2210116  0.2469688
64     0.0000000  0.0000000
65     0.0000000  0.0000000
66     0.0000000  0.0000000
67     0.0272484  0.0304487
68     0.2424463  0.2709208
69     0.3847957  0.4299888
70     0.1016530  0.1135919
71     0.0527057  0.0588958
72     0.1862758  0.2081533
73     0.0000000  0.0000000
74     0.0000000  0.0000000
75     0.0324162  0.0362234
76     0.0039052  0.0043639
77     0.1688344  0.1886635
78     0.0383768  0.0428840
79     0.0000000  0.0000000
80     0.7762272  0.8673929
81     0.2096483  0.2342709
82     0.0000000  0.0000000
83     0.0281293  0.0314330
84     0.1304871  0.1458123
85     0.0000000  0.0000000
86     0.0000000  0.0000000
87     0.2535159  0.2832906
88     0.0300085  0.0335329
89     0.0000000  0.0000000
90     0.0000000  0.0000000
91     0.0000000  0.0000000
92     0.0218164  0.0243786
93     0.0000000  0.0000000
94     0.2979414  0.3329337
95     0.0096309  0.0107620
96     0.0000000  0.0000000
97     0.3478576  0.3887125
98     0.0000000  0.0000000
99     0.0000000  0.0000000
100    0.0009103  0.0010171
101    0.0000000  0.0000000
102    0.0009983  0.0011156
103    0.0000000  0.0000000
104    0.1453445  0.1624147
105    0.3396948  0.3795910
106    0.2766242  0.3091129
107    0.1715064  0.1916493
108    0.1253192  0.1400376
109    0.0097777  0.0109261
110    0.0000000  0.0000000
111    0.4194728  0.4687386
112    0.0000000  0.0000000
113    0.0329741  0.0368468
114    0.0114808  0.0128291
115    0.0000000  0.0000000
116    0.0045512  0.0050857
117    0.1233225  0.1378064
118    0.0000000  0.0000000
119    0.0136242  0.0152243
120    0.1232051  0.1376752
121    0.3580464  0.4000979
122    0.0000000  0.0000000
123    0.0000000  0.0000000
124    0.0000000  0.0000000
125    0.0000000  0.0000000
126    0.0000000  0.0000000
127    0.0045512  0.0050857
128    0.0000000  0.0000000
129    0.0000000  0.0000000
130    0.0000000  0.0000000
131    0.0000000  0.0000000
132    0.0738174  0.0824870
133    0.1362127  0.1522105
134    0.1882724  0.2103844
135    0.0000000  0.0000000
136    0.4836298  0.5404308
137    0.0732301  0.0818308
138    0.0000000  0.0000000
139    0.0000000  0.0000000
140    0.0622779  0.0695922
141    0.0364389  0.0407185
142    0.23

In [ ]:
# 0.001 (1e-3)-> 
# 0.003 (3e-3)-> 
# 0.01 (1e-2)-> 
# 0.03 (3e-2)-> 
# 0.1 (1e-1)-> 
# 0.3 (3e-1)-> 
# 1

In [71]:
payment = x_raw

In [77]:
payment[payment['amount_spent_pay']==0].count()

Unnamed: 0                 16274
acc_id                     16274
playtime                   16274
npc_kill                   16274
solo_exp                   16274
party_exp                  16274
quest_exp                  16274
boss_monster               16274
death                      16274
revive                     16274
exp_recovery               16274
fishing                    16274
private_shop               16274
game_money_change          16274
enchant_count              16274
level                      16274
pledge_cnt                 16274
random_attacker_cnt        16274
random_defender_cnt        16274
temp_cnt                   16274
same_pledge_cnt            16274
etc_cnt                    16274
num_opponent               16274
amount_spent_pay           16274
sell_item_cnt              16274
buy_item_cnt               16274
play_char_cnt              16274
combat_char_cnt            16274
pledge_combat_cnt          16274
random_attacker_cnt_plg    16274
random_def

In [122]:
payment.describe()[['amount_spent_pay']]

amount_spent_pay
count      40000.000000
mean           1.109743
std            1.858979
min            0.000000
25%            0.000000
50%            0.348521
75%            1.310766
max           12.556131

In [156]:
x.describe()[['amount_spent_pay']] # 0원 제외

amount_spent_pay
count      23726.000000
mean           1.870931
std            2.098117
min            0.011735
25%            0.371990
50%            1.056123
75%            2.461911
max           12.556131

In [150]:
from scipy.stats import describe
describe(x)

DescribeResult(nobs=23726, minmax=(array([ 2.37012087e-03,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00, -8.90657004e+01,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  1.17347016e-02,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00]), array([  92.94903006,  334.39257588,  238.79182324,  300.14545253,
       1146.30416302,   99.        ,  614.6512978 ,  617.7887388 ,
        286.47446016,   84.31456123,   92.5300717 ,  200.16364807,
        565.34453953,   17.        ,  637.15697651,  826.55677045,
        154.98144752,  169.34390078, 

In [153]:
np.percentile(x[['amount_spent_pay']], 25) # 1사분위수

0.3719900391996848

In [154]:
np.percentile(x[['amount_spent_pay']], 50)  # 2사분위수

1.0561231396836477

In [157]:
np.percentile(x[['amount_spent_pay']], 75)  # 3사분위수

2.4619110488642235

In [152]:
np.percentile(x[['amount_spent_pay']], 100)  # 최대

12.556130660683367